In [ ]:
import copy
import numpy as np
import ipynb.fs.full.misc as m

In [ ]:
class Breeder(object):
    def __init__(self,
                 population,
                 environment,
                 target_fitness,
                 generations=100,
                 elite_percent=0.1,
                 mutation_rate=0.1,
                 selection_pressure=0.1,
                 selection_func=None,
                 crossover_func=None,
                 mutation_func=None,
                 save_path="",
                 render=False):

        self.copy_to_save = None
        self.population = population
        self.environment = environment
        self.generations = generations
        self.target_fitness = target_fitness
        
        self.selection_func = selection_func
        self.crossover_func = crossover_func
        self.mutation_func = mutation_func
        self.mutation_rate = mutation_rate

        self.elite_percent = elite_percent
        self.selection_pressure = selection_pressure
        self.max_fitness = -1000000000.0
        self.do_render = render

        if population is not None:
            self.population_size = len(population)
        else:
            self.population_size = 0

        if hasattr(environment, 'env'):
            self.env_name = type(environment.env).__name__
        else:
            self.env_name = type(environment).__name__
        self.save_path = save_path

In [ ]:
class Breeder(Breeder):
    def selection(self, selection_pressure):
        return self.selection_func(self.population, selection_pressure)

    def crossover(self, parent_a, parent_b):
        assert parent_a.size == parent_b.size
        return self.crossover_func(parent_a, parent_b)

    def mutate(self, parent, mutation_rate):
        rate = 0.0
        if isinstance(mutation_rate, tuple):
            rate = np.random.uniform(mutation_rate[0], mutation_rate[1])
        else:
            rate = mutation_rate
        return self.mutation_func(parent, rate)

In [ ]:
class Breeder(Breeder):
    def update_fitness(self):
        for i in range(self.population_size): 
            self.population[i].evaluate(self.environment, render=self.do_render, sleep=False)

        self.population = sorted(self.population, key=lambda x: x.fitness, reverse=True)

In [ ]:
class Breeder(Breeder):
    def save_checkpoint(self):
        m.save_object(self.copy_to_save, self.save_path + self.env_name + "_checkpoint.pickle")

    def load_checkpoint(self):
        self.population = m.load_object(self.save_path + self.env_name + "_checkpoint.pickle")
        self.population_size = len(self.population)

In [ ]:
class Breeder(Breeder):
    def evolve(self):
        offsprings = self.population

        for g in range(0, self.generations):
            for p in offsprings:  # store only the last experiences in replay
                p.observation_buffer = None
                p.action_buffer = None

            self.copy_to_save = copy.deepcopy(offsprings)
            self.population = copy.deepcopy(offsprings)

            self.update_fitness()
            self.save_checkpoint()
            self.print_stats(g)

            if self.terminate():
                break

            elite_number = int(self.population_size * self.elite_percent)
            print("en", elite_number)
            for i in range(0, elite_number):
                offsprings[i] = self.population[i]  # the wise ones
                offsprings[i].age += 1

                
            # The way to go out from local minima
            # If there is no progress, we toss the best individual away
            if offsprings[0].age > 20: # revolution!
                print("Revolution time!")
                index = np.random.randint(0, self.population_size)
                offsprings[0] = self.population[index]

            for p in range(elite_number, self.population_size):
                partner_a, partner_b = self.selection(self.selection_pressure)

                child = self.crossover(partner_a, partner_b)
                child = self.mutate(child, self.mutation_rate)

                offsprings[p] = child

        return self.population[2]

In [ ]:
class Breeder(Breeder):
    def terminate(self):
        return self.population[0].fitness >= self.target_fitness

    def print_stats(self, generation):
        average_fitness = sum(agent.fitness for agent in self.population) / self.population_size

        print("Gen:", generation,
              ", Unadjusted max fitness:", self.population[0].unadjusted_fitness,
              ", Max fitness:", self.population[0].fitness,
              ", Min fitness:", self.population[-1].fitness,
              ", Age:", self.population[0].age,
              ", Mean fitness:", average_fitness)